# SQL TEa

In [19]:
import pg8000
import pandas as pd
pd.set_option('display.max_columns', None)

In [20]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results

#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

### Examples

In [21]:
my_query = """SELECT * 
              FROM specimens 
              LIMIT 1"""
#asterisk(*) = all columns
#specimens = table name
#LIMIT 10 = limits search to first 10 rows

my_result = limsquery(my_query)
first_element = my_result[0]
#first element is created to only look at the first row
#print my_result
print first_element.keys()
#Shows all columns and one row of entry

['cell_depth', 'ephys_roi_result_id', 'parent_y_coord', 'reference_space_id', 'updated_at', 'cell_label', 'preparation_method_id', 'parent_x_coord', 'task_id', 'location_id', 'id', 'cortex_layer_id', 'plane_of_section_id', 'frozen_at', 'flipped_specimen_id', 'data', 'pinned_radius', 'rna_integrity_number', 'histology_well_name', 'created_by', 'priority', 'parent_id', 'ephys_start_time_sec', 'project_id', 'alignment3d_id', 'carousel_well_name', 'patched_cell_container', 'updated_by', 'cell_prep_id', 'biophysical_model_state', 'barcode', 'storage_directory', 'x_coord', 'tissue_ph', 'specimen_preparation_method_id', 'donor_id', 'starter_cell_count', 'operation_id', 'ephys_neural_tissue_plan_id', 'structure_id', 'parent_z_coord', 'facs_well_id', 'name', 'y_coord', 'normalization_group_id', 'postmortem_interval_id', 'specimen_set_id', 'created_at', 'tissue_processing_id', 'ephys_cell_plan_id', 'hemisphere_id', 'ephys_qc_result', 'specimen_metadata_id', 'cell_reporter_id', 'task_flow_id', 'e

### Trial for SQL Set-up

In [22]:
lims_query = """SELECT *
                FROM specimens
                ORDER BY updated_at DESC
                LIMIT 10"""
lims_df = get_lims_dataframe(lims_query)
lims_df.head(10)

,cell_depth,ephys_roi_result_id,parent_y_coord,reference_space_id,updated_at,cell_label,preparation_method_id,parent_x_coord,task_id,location_id,id,cortex_layer_id,plane_of_section_id,frozen_at,flipped_specimen_id,data,pinned_radius,rna_integrity_number,histology_well_name,created_by,priority,parent_id,ephys_start_time_sec,project_id,alignment3d_id,carousel_well_name,patched_cell_container,updated_by,cell_prep_id,biophysical_model_state,barcode,storage_directory,x_coord,tissue_ph,specimen_preparation_method_id,donor_id,starter_cell_count,operation_id,ephys_neural_tissue_plan_id,structure_id,parent_z_coord,facs_well_id,name,y_coord,normalization_group_id,postmortem_interval_id,specimen_set_id,created_at,tissue_processing_id,ephys_cell_plan_id,hemisphere_id,ephys_qc_result,specimen_metadata_id,cell_reporter_id,task_flow_id,external_specimen_name
0,None,None,0.0,NaN,2019-04-10 19:50:22.691483,None,None,0.0,None,None,849056819,None,5,NaT,561557765,None,None,None,None,None,None,849056672.0,None,811632510,None,None,PXS4_190410_301_A01,None,None,review_required,0849056819,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11.03.01.01,None,None,None,None,2019-04-10 19:50:22.514947,None,None,None,None,None,NaN,None,None
1,None,None,0.0,NaN,2019-04-10 19:49:48.064979,None,None,0.0,None,None,849056672,None,5,NaT,561557765,None,None,None,None,None,None,849056625.0,None,811632510,None,None,None,None,None,review_required,0849056672,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11.03.01,None,None,None,None,2019-04-10 19:49:47.962573,None,None,None,None,None,NaN,None,None
2,None,None,0.0,NaN,2019-04-10 19:49:30.606923,None,None,0.0,None,None,849056625,None,5,NaT,561557765,None,None,None,None,None,None,849052715.0,None,811632510,None,None,None,None,None,review_required,0849056625,None,None,None,NaN,849052252,None,None,None,None,3.0,None,H19.03.307.11.03,None,None,None,None,2019-04-10 19:49:30.505119,None,None,None,None,None,NaN,None,None
3,None,None,NaN,9.0,2019-04-10 19:47:17.665366,None,None,NaN,None,None,786191814,None,1,2019-01-03 20:00:00,561557765,None,None,None,None,None,None,NaN,None,654688276,None,None,None,None,None,review_required,0786191814,None,None,None,283.0,786191809,None,None,None,None,NaN,None,Ctgf-T2A-dgCre-423064,None,None,None,None,2018-11-27 21:57:26.658584,None,None,None,None,None,NaN,None,423064
4,None,None,0.0,NaN,2019-04-10 19:46:40.483311,None,None,0.0,None,None,849053665,None,5,NaT,561557765,None,None,None,None,None,None,849052940.0,None,811632510,None,None,PXS4_190409_301_A01,None,None,review_required,0849053665,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11.01.01.01,None,None,None,None,2019-04-10 19:46:19.612416,None,None,None,None,None,491913822.0,None,None
5,None,None,0.0,NaN,2019-04-10 19:45:35.392447,None,None,0.0,None,None,849052940,None,5,NaT,561557765,None,None,None,None,None,None,849052884.0,None,811632510,None,None,None,None,None,review_required,0849052940,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11.01.01,None,None,None,None,2019-04-10 19:45:35.296335,None,None,None,None,None,NaN,None,None
6,None,None,0.0,NaN,2019-04-10 19:45:12.914936,None,None,0.0,None,None,849052884,None,5,NaT,561557765,None,None,None,None,None,None,849052715.0,None,811632510,None,None,None,None,None,review_required,0849052884,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11.01,None,None,None,None,2019-04-10 19:45:12.820167,None,None,None,None,None,NaN,None,None
7,None,None,0.0,NaN,2019-04-10 19:44:04.142317,None,None,0.0,None,None,849052715,None,5,NaT,561557765,None,None,None,None,None,None,849052651.0,None,811632510,None,None,None,None,None,review_required,0849052715,None,None,None,NaN,849052252,None,None,None,None,1.0,None,H19.03.307.11,None,None,None,None,2019-04-10 19:44:04.046879,None,None,None,None,None,NaN,None,None
8,None,None,NaN,NaN,2019-04-10 19:43:34.621432,None,None,NaN,None,None,849052651,None,5,2019-04-10

In [24]:
lims_query = """SELECT created_at, name, cell_depth, ephys_roi_result_id, patched_cell_container
                FROM specimens
                WHERE patched_cell_container is NOT NULL AND name is NOT NULL AND cell_depth is NOT NULL
                ORDER BY created_at DESC
                LIMIT 50"""
lims_df = get_lims_dataframe(lims_query)
lims_df.head(50)

,cell_depth,ephys_roi_result_id,created_at,patched_cell_container,name
0,0.0,848680960,2019-04-10 01:36:02.208746,PXS4_190409_655_A01,H19.06.352.99.02.03.01.05
1,0.0,848672017,2019-04-10 00:45:57.747369,PXS4_190409_654_A01,H19.06.352.99.02.03.01.04
2,0.0,848656880,2019-04-10 00:06:00.874210,PXS4_190409_653_A01,H19.06.352.99.02.03.01.03
3,47.0,848653368,2019-04-10 00:00:59.952518,PXS4_190409_360_A01,H19.06.352.99.01.06.01.10
4,49.3,848646547,2019-04-09 23:40:58.259406,PXS4_190409_359_A01,H19.06.352.99.01.06.01.09
5,37.7,848639558,2019-04-09 23:10:57.781218,PFS4_190409_705_A01,Gad2-IRES-Cre;Ai14-455016.03.01.02
6,68.6,848636862,2019-04-09 22:55:59.568331,PXS4_190409_358_A01,H19.06.352.99.01.06.01.08
7,39.4,848636058,2019-04-09 22:50:58.176535,PAS4_190409_455_A01,Gad2-IRES-Cre;Ai14-455016.04.02.01
8,48.9,848635200,2019-04-09 22:45:57.915468,PFS4_190409_704_A01,Gad2-IRES-Cre;Ai14-455016.03.01.01
9,70.7,848632801,2019-04-09 22:30:58.961107,PXS4_190409_357_A01,H19.06.352.99.01.06.01.07
